# Example RAG with Opik Tracing and Evals

This simple example will get you started with using Opik, Weaviate, and the OpenAI API to build a RAG system.


# Set up your Environment

[Comet](https://www.comet.com/) provides a hosted version of the Opik platform, simply [create a free account](https://www.comet.com/site/products/opik/) and grab you API Key from the UI.

First, we need pip install the opik and openai libraries.

In [ ]:
%pip install -U opik openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.5/148.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.0/400.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.2/473.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 5.5 MB/s eta 0:00:00


Now, we'll configure Opik and OpenAI with our respective API keys.

In [ ]:
import opik

opik.configure(use_local=False)

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "statisticianinstilettos" workspace? (Y/n)Y


OPIK: Configuration saved to file: /root/.opik.config


In [ ]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


Traces will now be automatically logged to the Opik UI where you can inspect the inputs, outputs, and configure evaluation metrics. After you run this cell, follow the link to the Comet UI to see you traces.

# Set up Weaviate Client

Weaviate is a vector database which supports billion scale vector search with sub 50ms query times. We'll use Weaviate to query for books in this example.

In [ ]:
%pip install -U weaviate-client --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.3/353.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 24.7 MB/s eta 0:00:00


In [ ]:
import os
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.init import AdditionalConfig, Timeout


WEAVIATE_CLUSTER_URL = os.getenv('WEAVIATE_CLUSTER_URL') or 'https://zxzyqcyksbw7ozpm5yowa.c0.us-west2.gcp.weaviate.cloud'
WEAVIATE_API_KEY = os.getenv('WEAVIATE_API_KEY') or 'n6mdfI32xrXF3DH76i8Pwc2IajzLZop2igb6' # This is a read key

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_CLUSTER_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
    headers={"X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]},
)

print(weaviate_client.is_connected())

book_collection = weaviate_client.collections.get(name="Book")

True


# Write a RAG app with OpenAI, Weaviate and Opik Traces

Next, we will build a very simple LLM reasoning application and log the trace data to Opik where we can apply additional evaluation metrics and debug the LLM response.

We will use Opik to collect traces to inspect the inputs and outputs of the reasoning tasks, and to create evaluation metrics for hallicinations and other common or custom issues you want to detect.

Opik integrates with OpenAI to provide a simple way to log traces for all OpenAI LLM calls. This works for all OpenAI models, including if you are using the streaming API.

In [ ]:
from opik.integrations.openai import track_openai
from openai import OpenAI

os.environ["OPIK_PROJECT_NAME"] = "rag-project" #name your project. This will appear as the project name in the Opik UI


client = OpenAI()
client = track_openai(client)

We are using the @opik.track decorator and the OpenAI logging integration to automatically log our traces and spans. Learn more here https://www.comet.com/docs/opik/tracing/log_traces#using-an-integration

In [ ]:
@opik.track
def retrieve_context(user_query):
    # Semantic Search
    response = book_collection.query.near_text(
        query=user_query,
        limit=3
    )

    recommended_books = []
    for book in response.objects:
        recommended_books.append(book.properties['title'])
    return recommended_books

In [ ]:
@opik.track
def generate_response(user_query, recommended_books):
  prompt = f"""
  You're a helpful assistant, reply to a chatbot message for someone inquiring for
  book recommendations. The user query was {user_query}


  These were the book that were extracted from the vector
  search:

  {recommended_books}
  """

  response = client.chat.completions.create(
      model="o3-mini",
      messages=[
          {
              "role": "user",
              "content": prompt
          }
      ]
  )

  return (response.choices[0].message.content)

In [ ]:
@opik.track(name="rag-example")
def llm_chain(user_query):
    context = retrieve_context(user_query)
    response = generate_response(user_query, context)
    return response

In [ ]:
# Use the LLM chain
user_query = input("What types of books are you looking for? ")
result = llm_chain(user_query)
print(result)

What types of books are you looking for? running


OPIK: Started logging traces to the "rag-project" project at https://www.comet.com/opik/statisticianinstilettos/redirect/projects?name=rag-project.


Based on your interest in running, here are three recommendations we found:

• Running in the Family – Although known primarily as Michael Ondaatje’s vivid memoir about family history, this book uses running as a kind of metaphor for life’s journeys and the passage of time. Its lyrical style might resonate if you enjoy reflective, meandering narratives.

• Winterdance – Also subtitled “The Fine Madness of Running the Iditarod,” this memoir by Gary Paulsen offers an exhilarating and personal account of the brutal, yet deeply captivating, world of dog sled racing. If you’re drawn to stories of endurance and adventure, this one delivers a firsthand glimpse into one of the most challenging races on Earth.

• Drina Dances Again – While perhaps a less obvious connection to “running,” this title suggests a narrative filled with movement and energy. Often, such works blend historical or cultural insights with an underlying pulse of forward momentum. It might offer a unique literary journey if 